In [16]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable
from biogeme import models
from biogeme import results as res

Reading and preparing the data

In [17]:
#Import the data into Pandas
df=pd.read_csv('/Applications/epfl/epfl3/Mathematics modeling of behavior/project/lpmc12.dat',sep='\t')
df

,trip_id,household_id,person_n,trip_n,travel_mode,purpose,fueltype,faretype,bus_scale,survey_year,...,dur_pt_access,dur_pt_rail,dur_pt_bus,dur_pt_int,pt_interchanges,dur_driving,cost_transit,cost_driving_fuel,cost_driving_ccharge,driving_traffic_percent
0,1,0,0,1,4,3,1,1,1.0,1,...,0.109444,0.000000,0.055556,0.000000,0,0.059444,1.5,0.15,0.0,0.112150
1,13,1,1,1,4,3,1,5,0.0,1,...,0.241389,0.000000,0.122222,0.000000,0,0.132222,0.0,0.50,0.0,0.065126
2,19,4,0,1,3,3,6,5,0.0,1,...,0.222500,0.000000,0.312222,0.000000,0,0.221667,0.0,0.56,0.0,0.086466
3,20,5,1,0,4,3,1,5,0.0,1,...,0.381667,0.000000,0.062222,0.000000,0,0.117222,0.0,0.41,0.0,0.097156
4,39,9,2,0,4,3,1,5,0.0,1,...,0.146944,0.000000,0.225000,0.000000,0,0.200833,0.0,0.48,0.0,0.378976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,81035,17603,1,1,3,1,1,1,1.0,3,...,0.278056,0.216667,0.000000,0.000000,0,0.392222,2.4,0.98,0.0,0.547450
4996,81040,17604,2,0,3,1,1,2,0.0,3,...,0.264444,0.000000,0.353333,0.176667,1,0.288889,0.0,0.85,0.0,0.175000
4997,81045,17605,0,1,4,3,1,5,0.0,3,...,0.128889,0.000000,0.045833,0.000000,0,0.067778,0.0,0.17,0.0,0.024590
4998,81066,17608,0,3,3,3,6,1,1.0,3,...,0.092222,0.000000,0.389444,0.000000,0,0.193333,1.5,0.61,0.0,0.485632


In [18]:
#Import this database into Biogeme.
database=db.Database('/Applications/epfl/epfl3/Mathematics modeling of behavior/project/lpmc12.dat',df)

In [19]:
#We identify the columns that will be used as variables in our model. 
#cost
cost_driving_fuel=Variable('cost_driving_fuel')
cost_driving_ccharge=Variable('cost_driving_ccharge')
cost_transit=Variable('cost_transit')
#time
dur_driving=Variable('dur_driving')
dur_walking=Variable('dur_walking')
dur_cycling=Variable('dur_cycling')
dur_pt_access=Variable('dur_pt_access')
dur_pt_rail=Variable('dur_pt_rail')
dur_pt_bus=Variable('dur_pt_bus')
dur_pt_int=Variable('dur_pt_int')#how to consider this one
#mode
travel_mode= Variable('travel_mode')

In [20]:
#The total travel time by public transport is the sum of the in-vehicle travel time
pt_time=dur_pt_rail+dur_pt_access+dur_pt_bus+dur_pt_int

In [21]:
#The total travel time by walking is the sum of the walking travel time
walking_time=dur_walking

In [22]:
#The total cycling time by cycling is the sum of the cycling travel time
cycling_time=dur_cycling

In [23]:
#The total driving time by driving is the sum of the driving travel time
driving_time=dur_driving

In [24]:
#The cost section of driving
driving_cost=cost_driving_ccharge+cost_driving_ccharge

In [25]:
#The cost section of pt
pt_cost=cost_transit

Model with constant only


We are now ready to specify the choice model. We start with a simple model, that contains only one alternative specific constant
\begin{align*}
V_\text{driving} &= \text{ASC}_\text{driving}, \\ V_\text{pt} &= \text{ASC}_\text{pt}, \\  V_\text{walking} &= \text{ASC}_\text{walking}, \\V_\text{cycling}&=0
\end{align*}

We define the unknown parameter using the Biogeme expression `Beta`, that takes 5 parameters:
- the name of the parameter (it is advised to use the exact same name for the corresponding Python variable),
- the starting value for the estimation (usually, 0),
- a lower bound on the value of the coefficient, or `None` for no bound,
- an upper bound, or `None`for no bound,
- a parameter that is 1 if the value of the parameter must be fixed to its starting value, and 0 if it has to be estimated.

In [26]:
asc_driving=Beta('asc_driving',0,None,None,0)
asc_pt=Beta('asc_pt',0,None,None,0)
asc_walking=Beta('asc_walking',0,None,None,0)

Define the generatic parameters

In [27]:
beta_time=Beta('beta_time',0,None,None,0)


beta_cost=Beta('beta_cost',0,None,None,0)




In [28]:
#We now write the utility functions
Opt1_driving = (
    asc_driving+
    beta_time * driving_time
    + beta_cost * driving_cost
)
Opt2_pt = (
    asc_pt
    + beta_time * pt_time
    + beta_cost * pt_cost
)
Opt3_cycling = (
    beta_time*cycling_time
)
Opt4_walking = (
    asc_walking+beta_time*walking_time
)

V_base = {1: Opt1_driving, 2: Opt2_pt, 3: Opt3_cycling, 4:Opt4_walking}

# The choice model is a logit, with availability conditions
logprob_base = models.loglogit(V_base, None, travel_mode)
biogeme_base = bio.BIOGEME(database, logprob_base)
biogeme_base.modelName = 'model0_base'
all_results = {}
all_results['Base'] = biogeme_base.estimate()

In [29]:
#And we write the choice model(how for my case)
res.compileEstimationResults(all_results)

,Base
Number of estimated parameters,5.000000
Sample size,5000.000000
Final log likelihood,-5704.233959
Akaike Information Criterion,11418.467917
Bayesian Information Criterion,11451.053883
asc_driving,-0.709475
asc_pt,-2.409886
asc_walking,0.149816
beta_cost,-0.016374
beta_time,0.046923


In [30]:
results = biogeme_base.estimate()
print(results.getEstimatedParameters())

                Value  Rob. Std err  Rob. t-test  Rob. p-value
asc_driving -0.709475      0.042679   -16.623650      0.000000
asc_pt      -2.409886      0.082702   -29.139230      0.000000
asc_walking  0.149816      0.038205     3.921421      0.000088
beta_cost   -0.016374      0.006445    -2.540634      0.011065
beta_time    0.046923      0.034001     1.380035      0.167576
